<a href="https://colab.research.google.com/github/mritunjay29-ai/Namekart-Domain-Name-Evaluation/blob/main/Namekart_Price_Model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import BertModel

In [ ]:
df = pd.read_csv("/content/data.csv")

In [ ]:
price_features = ['bidders', 'sld_length', 'encoded_category']
bid_features = ['price']

target_price = 'price'
target_bid = 'bid_amount'

X_price = df[price_features].values
X_bid = df[bid_features].values
y_price = df[target_price].values
y_bid = df[target_bid].values

X_price_train, X_price_test, X_bid_train, X_bid_test, y_price_train, y_price_test, y_bid_train, y_bid_test = train_test_split(
    X_price, X_bid, y_price, y_bid, test_size=0.2, random_state=42
)

scaler_price = StandardScaler()
scaler_bid = StandardScaler()

X_price_train = scaler_price.fit_transform(X_price_train)
X_price_test = scaler_price.transform(X_price_test)
X_bid_train = scaler_bid.fit_transform(X_bid_train)
X_bid_test = scaler_bid.transform(X_bid_test)

X_price_train = torch.tensor(X_price_train, dtype=torch.float32)
X_price_test = torch.tensor(X_price_test, dtype=torch.float32)
X_bid_train = torch.tensor(X_bid_train, dtype=torch.float32)
X_bid_test = torch.tensor(X_bid_test, dtype=torch.float32)
y_price_train = torch.tensor(y_price_train, dtype=torch.float32).view(-1, 1)
y_price_test = torch.tensor(y_price_test, dtype=torch.float32).view(-1, 1)
y_bid_train = torch.tensor(y_bid_train, dtype=torch.float32).view(-1, 1)
y_bid_test = torch.tensor(y_bid_test, dtype=torch.float32).view(-1, 1)

class BERTMultiTaskModel(nn.Module):
    def __init__(self, price_dim, bid_dim):
        super(BERTMultiTaskModel, self).__init__()

        self.bert = BertModel.from_pretrained("bert-base-uncased")

        self.price_fc = nn.Linear(price_dim, 768)
        self.bid_fc = nn.Linear(bid_dim, 768)

        self.price_out = nn.Linear(768, 1)
        self.bid_out = nn.Linear(768, 1)

    def forward(self, x_price, x_bid):

        x_price_embed = self.price_fc(x_price)
        x_bid_embed = self.bid_fc(x_bid)

        x_price_bert = self.bert(inputs_embeds=x_price_embed.unsqueeze(1)).last_hidden_state[:, 0, :]
        x_bid_bert = self.bert(inputs_embeds=x_bid_embed.unsqueeze(1)).last_hidden_state[:, 0, :]

        price_output = self.price_out(x_price_bert)
        bid_output = self.bid_out(x_bid_bert)

        return price_output, bid_output

input_price_dim = X_price_train.shape[1]
input_bid_dim = X_bid_train.shape[1]
model = BERTMultiTaskModel(input_price_dim, input_bid_dim)

criterion = nn.MSELoss()
optimizer =
num_epochs = 20
losses = []

def train_model(model, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, criterion, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        price_pred, bid_pred = model(X_price_train, X_bid_train)
        loss_price = criterion(price_pred, y_price_train)
        loss_bid = criterion(bid_pred, y_bid_train)
        total_loss = loss_price + loss_bid
        total_loss.backward()
        optimizer.step()

        losses.append(total_loss.item())
        if epoch % 5 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss.item()}')

    print("Training Complete")

train_model(model, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, criterion, num_epochs)

def evaluate_and_plot(model, X_price_test, X_bid_test, y_price_test, y_bid_test):
    model.eval()
    with torch.no_grad():
        price_pred, bid_pred = model(X_price_test, X_bid_test)

    plt.figure(figsize=(10, 5))
    plt.plot(y_price_test.numpy(), label='Actual Price', color='blue')
    plt.plot(price_pred.numpy(), label='Predicted Price', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Price')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(y_bid_test.numpy(), label='Actual Bid Amount', color='blue')
    plt.plot(bid_pred.numpy(), label='Predicted Bid Amount', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Bid Amount')
    plt.show()

evaluate_and_plot(model, X_price_test, X_bid_test, y_price_test, y_bid_test)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
X_price_train, X_price_test, X_bid_train, X_bid_test, y_price_train, y_price_test, y_bid_train, y_bid_test = train_test_split(
    X_price, X_bid, y_price, y_bid, test_size=0.2, random_state=42
)
scaler_price = StandardScaler()
scaler_bid = StandardScaler()

X_price_train = scaler_price.fit_transform(X_price_train)
X_price_test = scaler_price.transform(X_price_test)
X_bid_train = scaler_bid.fit_transform(X_bid_train)
X_bid_test = scaler_bid.transform(X_bid_test)

X_price_train = torch.tensor(X_price_train, dtype=torch.float32)
X_price_test = torch.tensor(X_price_test, dtype=torch.float32)
X_bid_train = torch.tensor(X_bid_train, dtype=torch.float32)
X_bid_test = torch.tensor(X_bid_test, dtype=torch.float32)
y_price_train = torch.tensor(y_price_train, dtype=torch.float32).view(-1, 1)
y_price_test = torch.tensor(y_price_test, dtype=torch.float32).view(-1, 1)
y_bid_train = torch.tensor(y_bid_train, dtype=torch.float32).view(-1, 1)
y_bid_test = torch.tensor(y_bid_test, dtype=torch.float32).view(-1, 1)

In [ ]:
class TransformerMultiTaskModel(nn.Module):
    def __init__(self, price_dim, bid_dim):
        super(TransformerMultiTaskModel, self).__init__()

        self.price_proj = nn.Linear(price_dim, 128)
        self.bid_proj = nn.Linear(bid_dim, 128)

        self.shared_layer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=128, nhead=8), num_layers=2
        )

        self.price_fc1 = nn.Linear(128, 64)
        self.price_fc2 = nn.Linear(64, 32)
        self.price_out = nn.Linear(32, 1)

        self.bid_fc1 = nn.Linear(128, 64)
        self.bid_fc2 = nn.Linear(64, 32)
        self.bid_out = nn.Linear(32, 1)

    def forward(self, x_price, x_bid):

        x_price = self.price_proj(x_price)
        x_bid = self.bid_proj(x_bid)

        x_price = x_price.unsqueeze(0)
        x_bid = x_bid.unsqueeze(0)

        x_price = self.shared_layer(x_price).squeeze(0)
        x_bid = self.shared_layer(x_bid).squeeze(0)

        price_hidden = torch.relu(self.price_fc1(x_price))
        price_hidden = torch.relu(self.price_fc2(price_hidden))
        price_output = self.price_out(price_hidden)

        bid_hidden = torch.relu(self.bid_fc1(x_bid))
        bid_hidden = torch.relu(self.bid_fc2(bid_hidden))
        bid_output = self.bid_out(bid_hidden)

        return price_output, bid_output

In [ ]:
input_price_dim = X_price_train.shape[1]
input_bid_dim = X_bid_train.shape[1]
model = TransformerMultiTaskModel(input_price_dim, input_bid_dim)

mse_loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
losses = []

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
def train_model(model, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, mse_loss, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        price_pred, bid_pred = model(X_price_train, X_bid_train)
        loss_price = mse_loss(price_pred, y_price_train)
        loss_bid = mse_loss(bid_pred, y_bid_train)
        total_loss = loss_price + loss_bid
        total_loss.backward()
        optimizer.step()

        losses.append(total_loss.item())
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss.item()}')

    print("Training Complete")

train_model(model, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, mse_loss, num_epochs)

In [ ]:
def evaluate_and_plot(model, X_price_test, X_bid_test, y_price_test, y_bid_test):
    model.eval()
    with torch.no_grad():
        price_pred, bid_pred = model(X_price_test, X_bid_test)

    plt.figure(figsize=(10, 5))
    plt.plot(y_price_test.numpy(), label='Actual Price', color='blue')
    plt.plot(price_pred.numpy(), label='Predicted Price', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Price')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(y_bid_test.numpy(), label='Actual Bid Amount', color='blue')
    plt.plot(bid_pred.numpy(), label='Predicted Bid Amount', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Bid Amount')
    plt.show()

In [ ]:
evaluate_and_plot(model, X_price_test, X_bid_test, y_price_test, y_bid_test)

In [ ]:
df.head()

In [ ]:
shared_features = ['keyword_exact_cpc', 'sld_length', 'bidders']
price_specific_features = ['encoded_category', 'extensions_taken']
bid_specific_features = ['extensions_taken']


In [ ]:
target_price = 'price'
target_bid = 'bid_amount'

In [ ]:
X_shared = df[shared_features].values
X_price = df[price_specific_features].values
X_bid = df[bid_specific_features].values

In [ ]:
y_price = df[target_price].values
y_bid = df[target_bid].values

In [ ]:
X_shared_train, X_shared_test, X_price_train, X_price_test, X_bid_train, X_bid_test, y_price_train, y_price_test, y_bid_train, y_bid_test = train_test_split(
    X_shared, X_price, X_bid, y_price, y_bid, test_size=0.2, random_state=42
)

In [ ]:
scaler_shared = StandardScaler()
scaler_price = StandardScaler()
scaler_bid = StandardScaler()

In [ ]:
X_shared_train = scaler_shared.fit_transform(X_shared_train)
X_shared_test = scaler_shared.transform(X_shared_test)

X_price_train = scaler_price.fit_transform(X_price_train)
X_price_test = scaler_price.transform(X_price_test)

X_bid_train = scaler_bid.fit_transform(X_bid_train)
X_bid_test = scaler_bid.transform(X_bid_test)

In [ ]:
X_shared_train = torch.tensor(X_shared_train, dtype=torch.float32)
X_shared_test = torch.tensor(X_shared_test, dtype=torch.float32)
X_price_train = torch.tensor(X_price_train, dtype=torch.float32)
X_price_test = torch.tensor(X_price_test, dtype=torch.float32)
X_bid_train = torch.tensor(X_bid_train, dtype=torch.float32)
X_bid_test = torch.tensor(X_bid_test, dtype=torch.float32)

In [ ]:
y_price_train = torch.tensor(y_price_train, dtype=torch.float32).view(-1, 1)
y_price_test = torch.tensor(y_price_test, dtype=torch.float32).view(-1, 1)
y_bid_train = torch.tensor(y_bid_train, dtype=torch.float32).view(-1, 1)
y_bid_test = torch.tensor(y_bid_test, dtype=torch.float32).view(-1, 1)

In [ ]:
class MultiTaskModel(nn.Module):
    def __init__(self, shared_dim, price_dim, bid_dim):
        super(MultiTaskModel, self).__init__()

        self.shared_fc = nn.Linear(shared_dim, 128)
        self.shared_relu = nn.ReLU()

        self.price_fc1 = nn.Linear(128 + price_dim, 64)
        self.price_out = nn.Linear(64, 1)

        self.bid_fc1 = nn.Linear(128 + bid_dim, 64)
        self.bid_out = nn.Linear(64, 1)

    def forward(self, x_shared, x_price, x_bid):
        shared_features = self.shared_relu(self.shared_fc(x_shared))

        price_input = torch.cat((shared_features, x_price), dim=1)
        price_hidden = torch.relu(self.price_fc1(price_input))
        price_output = self.price_out(price_hidden)

        bid_input = torch.cat((shared_features, x_bid), dim=1)
        bid_hidden = torch.relu(self.bid_fc1(bid_input))
        bid_output = self.bid_out(bid_hidden)

        return price_output, bid_output

In [ ]:
shared_dim = X_shared_train.shape[1]
price_dim = X_price_train.shape[1]
bid_dim = X_bid_train.shape[1]

In [ ]:
model = MultiTaskModel(shared_dim, price_dim, bid_dim)

In [ ]:
mse_loss = nn.MSELoss()
kl_loss = nn.KLDivLoss(reduction='batchmean')
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 100
losses = []

In [ ]:
def train_model(model, X_shared_train, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        price_pred, bid_pred = model(X_shared_train, X_price_train, X_bid_train)
        loss_price = mse_loss(price_pred, y_price_train)
        loss_bid = mse_loss(bid_pred, y_bid_train)
        total_loss = loss_price + loss_bid
        total_loss.backward()
        optimizer.step()

        losses.append(total_loss.item())
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss.item()}')

    print("Training Complete")

In [ ]:
train_model(model, X_shared_train, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, num_epochs)

In [ ]:
def evaluate_and_plot(model, X_shared_test, X_price_test, X_bid_test, y_price_test, y_bid_test):
    model.eval()
    with torch.no_grad():
        price_pred, bid_pred = model(X_shared_test, X_price_test, X_bid_test)

    plt.figure(figsize=(10, 5))
    plt.plot(y_price_test.numpy(), label='Actual Price', color='blue')
    plt.plot(price_pred.numpy(), label='Predicted Price', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Price')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(y_bid_test.numpy(), label='Actual Bid Amount', color='blue')
    plt.plot(bid_pred.numpy(), label='Predicted Bid Amount', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Bid Amount')
    plt.show()

In [ ]:
evaluate_and_plot(model, X_shared_test, X_price_test, X_bid_test, y_price_test, y_bid_test)


In [ ]:
price_features = ['bidders', 'sld_length', 'encoded_category']
bid_features = ['price']

target_price = 'price'
target_bid = 'bid_amount'

X_price = df[price_features].values
X_bid = df[bid_features].values

y_price = df[target_price].values
y_bid = df[target_bid].values

X_price_train, X_price_test, X_bid_train, X_bid_test, y_price_train, y_price_test, y_bid_train, y_bid_test = train_test_split(
    X_price, X_bid, y_price, y_bid, test_size=0.2, random_state=42
)
scaler_price = StandardScaler()
scaler_bid = StandardScaler()

X_price_train = scaler_price.fit_transform(X_price_train)
X_price_test = scaler_price.transform(X_price_test)
X_bid_train = scaler_bid.fit_transform(X_bid_train)
X_bid_test = scaler_bid.transform(X_bid_test)

X_price_train = torch.tensor(X_price_train, dtype=torch.float32)
X_price_test = torch.tensor(X_price_test, dtype=torch.float32)
X_bid_train = torch.tensor(X_bid_train, dtype=torch.float32)
X_bid_test = torch.tensor(X_bid_test, dtype=torch.float32)
y_price_train = torch.tensor(y_price_train, dtype=torch.float32).view(-1, 1)
y_price_test = torch.tensor(y_price_test, dtype=torch.float32).view(-1, 1)
y_bid_train = torch.tensor(y_bid_train, dtype=torch.float32).view(-1, 1)
y_bid_test = torch.tensor(y_bid_test, dtype=torch.float32).view(-1, 1)

class MultiTaskModel(nn.Module):
    def __init__(self, price_dim, bid_dim):
        super(MultiTaskModel, self).__init__()

        self.price_fc1 = nn.Linear(price_dim, 64)
        self.price_fc2 = nn.Linear(64, 32)
        self.price_out = nn.Linear(32, 1)

        self.bid_fc1 = nn.Linear(bid_dim, 64)
        self.bid_fc2 = nn.Linear(64, 32)
        self.bid_out = nn.Linear(32, 1)

    def forward(self, x_price, x_bid):
        price_hidden = torch.relu(self.price_fc1(x_price))
        price_hidden = torch.relu(self.price_fc2(price_hidden))
        price_output = self.price_out(price_hidden)

        bid_hidden = torch.relu(self.bid_fc1(x_bid))
        bid_hidden = torch.relu(self.bid_fc2(bid_hidden))
        bid_output = self.bid_out(bid_hidden)

        return price_output, bid_output

input_price_dim = X_price_train.shape[1]
input_bid_dim = X_bid_train.shape[1]
model = MultiTaskModel(input_price_dim, input_bid_dim)

mse_loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
losses = []

In [ ]:
def train_model(model, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, mse_loss, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        price_pred, bid_pred = model(X_price_train, X_bid_train)
        loss_price = mse_loss(price_pred, y_price_train)
        loss_bid = mse_loss(bid_pred, y_bid_train)
        total_loss = loss_price + loss_bid
        total_loss.backward()
        optimizer.step()

        losses.append(total_loss.item())
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {total_loss.item()}')

    print("Training Complete")

train_model(model, X_price_train, X_bid_train, y_price_train, y_bid_train, optimizer, mse_loss, num_epochs)

In [ ]:
def evaluate_and_plot(model, X_price_test, X_bid_test, y_price_test, y_bid_test):
    model.eval()
    with torch.no_grad():
        price_pred, bid_pred = model(X_price_test, X_bid_test)

    plt.figure(figsize=(10, 5))
    plt.plot(y_price_test.numpy(), label='Actual Price', color='blue')
    plt.plot(price_pred.numpy(), label='Predicted Price', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Price')
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.plot(y_bid_test.numpy(), label='Actual Bid Amount', color='blue')
    plt.plot(bid_pred.numpy(), label='Predicted Bid Amount', color='red', linestyle='dashed')
    plt.legend()
    plt.title('Actual vs Predicted Bid Amount')
    plt.show()

evaluate_and_plot(model, X_price_test, X_bid_test, y_price_test, y_bid_test)